# Interfaces: From Protocols to ABCs

## Interfaces and Protocols in Python History

* Most existing code does not use them at all

- Reminder
     - Protocols are defined as the informal interfaces that make polymorphism work in dynamic languages
     
- Every class has an interface
     - Set public attributes (\__getitem\__) implemented by the class or super
     - By defintition, private and protected attributes are not part of the interface
     - An interface can be thought of the subset of public methods in a class that allow it to play a specifit role int he system
     
- Protcol
    - An interface seen as a set of methods to fulfill a role
    - A class may implement several protocols, each serving a different role
        - __Should it?__
    -  Protocols are informal interfaces so they cant be enforced

## Python Digs Sequences

* Sequences are one of the most fundamental interfaces in Python

- When pythons sees a \__getitem\__ method, it tries to iterate over the object
    - This means it also accepts the "in" operator even without a \__contains\__ method

## Monkey-Patching to Implement a Protocol at Runtime

- When following established protocols, chances of leveraging external libraries are improved

- random.shuffle gets a sequence as an input
    - For this to work a class needs to support item assignment
        - \__setitem\__ need to be implemented to make a sequence mutable
        
def set_card(deck, position, card):<br>
deck._cards[position] = card

FrenchDeck.__setitem__ = set_card<br>
shuffle(deck)

* Every Python method starts as a plain function, naming the first argument self is just a convention

- Monkey Patching
    - Changing a class or a module at runtime without touching the source code
   

## Alex Martelli's Waterfowl

- Avoid the use of isinstance to check the object's type since with "Duck Typing" we can see that an classes can be different but behave the same way

- Parallel evolution can often produce simialr traits, both morphological and behavioral among species that are unrelated but happened to evolve in similar enviroments and conditions

- Replacing "Duck typing" with "Goose Typing"
    - "Goose Typing" means that isnstance(obj, cls) should be used but only if cls is an abstract base class
    
- register class method
    - Lets end user declare that a certain class becomes a virtual subclass of an ABC
        - ABCs that boild down to just a couple simple methods dont requre to be registered


In [1]:
class Struggle:
    def __len__(self): return 23
    
from collections import abc
isinstance(Struggle(), abc.Sized)

True

- Whenever implementing a class that embodies any concepts repesented in the ABCs of any framework, subclass ir from or register it into the corresponding ABC
    - Inheriting or registering a class is also a decalration of intent by the programmer
    - Polymorphism shoudl be used to dispatch calls to proper methods instead of hardcoding the dispatch logic (n if loops)

## Subclassign and ABC

-  Subclassing an ABC such as collections.MutableSequence forces the programmer to implement certain methods such as \__delitem\__
    - Python does not check fro implementation of abstract methods at import time, just at runtime when the class is instantiated
    - Methods of ABCs can be overriden fro optimization purposes
        - For example \__contains\__ works by doing a full scan of the sequence but if our particular sequence is sorted there is no need to do a full scan


## ABCs in the Standard Library

- Most ABCs are defined in the collections.abc module

- Iterable/Container/Sized
    - All collections should should inherit or implement compatible protocols
    
- Sequence/Mapping/Set
    - Main immutable collection types, each with a mutable subclass
    
- MappingView
    - Objects returned from mapping methods - items(), keys(), values() return inherit from ItemsView, KeysView and ValuesView.
        - All inherit from the MappingView ABC, thus also inheriting from the Sized ABC
    
- Callable/Hashable
    - Main use is to support the isinstance built-in as a safe way of determining if an object is callable or hashable
    
    
## The Numbers Tower of ABCs

- The numbers package defines the "numerical tower"
    - Number is the topmost superclass
    - Complex is its immediate subclass
    - So on, down to Integral
   

## Defining and Using an ABC

- To define an ABC, subclass abc.ABC
- An abstract method is marked with the @abstractmethod decorator
    - Its body is often empty
- An abstract method can be implemented but it must still be overriden by the subclass
    - Subclass will be able to invoke the super method by using super()
    

## ABC Syntax Details

- The best way to decalre an ABC is to subset abc.ABC or any other ABC


## Subclassing the Tombola ABC

- It is possible to stack decorators on top of @abstractmethod

class MyABC(abc.ABC):<br>
    @classmethod<br>
    @abc.abstractmethod<br>
    def an_abstract_classmethod(cls):<br>
        pass
        
- Wer can just inherit suboptimal concrete methods (unnecessary to override) from an ABC, but program could be optimized by changes and tailoring methods to a specific implementation


## A Virtual Subclass of Tombola

- An characteristic of "Goose Typing" is the ability to register a class as a virtual subclass of ABC
    - The virual subclass wont actually inherit from the ABC
        - virtual subclass wont inherit any attributes or methods
    - The virtual class will be recognized as a subclass by __Tissubclass__ and __isinstance__

- Registering a Class
    - Done by calling a register method on the ABC
    - Invoked as a plain function or by using a decorator
    
- A class can be inspected with \__mro\__ 
    - A tuple of all real superclasses will be displayed

In [5]:
from random import randrange
from tombola import Tombola

@Tombola.register
class Tombolist(list):
    def pick(self):
        if self: #
            position = randrange(len(self))
            return self.pop(position) #
        else:
            raise LookupError('pop from empty TomboList')
    
    load = list.extend #
    
    def loaded(self):
        return bool(self) #
 
    def inspect(self):
        return tuple(sorted(self))

ModuleNotFoundError: No module named 'tombola'

## Usage of Register in Practice

- Register can be called as a plain function
     - Sequence.register(class)

## Geese can Behave as Ducks

- The \__subclasshook\__ adds some duck typing DNA to the whole goose typing proposition
    - Used for ABCs that implement just a low number of simple methods
    - Makes a program recognize subclasses of the ABC without inheritance or creating a virtual class